In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from ipywidgets import interact


df = pd.read_csv(r'C:\Users\Jouke\Documents\evedata-logger\output\market_data_with_names_merged.csv', parse_dates=['date'])

In [ ]:
# Get all unique item names
item_names = sorted(df['type_name'].dropna().astype(str).unique())


combobox = widgets.Combobox(
    options=item_names,
    placeholder='Search item',
    description='Item:',
    ensure_option=True,
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

display(combobox)


Combobox(value='', continuous_update=False, description='Item:', ensure_option=True, layout=Layout(width='400p…

In [ ]:
def predict_and_plot(selected_item):
    # Filter for the selected item
    df_item = df[df['type_name'] == selected_item].copy()
    df_item = df_item.sort_values('date').reset_index(drop=True)
    df_item['next_avg'] = df_item['average'].shift(-1)
    df_item = df_item.dropna(subset=['next_avg'])

    if len(df_item) < 10:
        print("Not enough data for this item!")
        return

    features = ['average', 'highest', 'lowest', 'volume']
    X = df_item[features]
    y = df_item['next_avg']

    split_idx = int(len(df_item) * 0.8)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Next Avg Price', marker='o')
    plt.plot(y_test.index, y_pred, label='Predicted Next Avg Price', marker='x')
    plt.xlabel('Test Set Index')
    plt.ylabel('Average Price')
    plt.title(f'Next Day Price Prediction: {selected_item}')
    plt.legend()
    plt.show()

    mae = mean_absolute_error(y_test, y_pred)
    print(f"Mean Absolute Error for {selected_item}: {mae:.2f}")

    r2 = r2_score(y_test, y_pred)
    print(f"R2 Score for {selected_item}: {r2:.4f}")

    comp = pd.DataFrame({'Actual': y_test.values, 'Predicted': y_pred})
    print(comp.head(10))

# Link the combobox for interactive prediction
interact(predict_and_plot, selected_item=combobox)


interactive(children=(Combobox(value='', continuous_update=False, description='Item:', ensure_option=True, lay…

<function __main__.predict_and_plot(selected_item)>